# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from konlpy.tag import Komoran
from gensim.models import Word2Vec

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')
df9 = pd.read_csv("data/babytimes.csv", encoding='utf-8')
df10 = pd.read_csv("data/daekyo.csv", encoding='utf-8')
df11 = pd.read_csv("data/mom&enfant.csv", encoding='utf-8')
df12 = pd.read_csv("data/namyang.csv", encoding='utf-8')

### Concatenate Dataframe

In [3]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text,
                df6.text, df7.text, df8.text, df9.text, df10.text,
                df11.text, df12.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
27050,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
27051,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
27052,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...
27053,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...
27054,임신 기간을 설명할 때 헷갈리는 경우가 많다 책을 볼 때 의사의 설명을 들을 때 정...


### Preprocessing

In [4]:
def cleaningText(text):
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+>|【.+】|\[.+\]', ' ', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', ' ', text)
    text = re.sub('[㈜×–～→▪‧•∘✽○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤]', ' ', text)
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', ' ', text)
    text = re.sub('[➊❷➌❹❺❻❼❽❾❿]', ' ', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net', ' ', text)
    text = re.sub("\s.+\s기자", '', text)
    text = re.sub('ⓒ\S+\s|㈜\S+\s|홈페이지\S+\s|전화\S+\s|대표전화\S+\s|☎\S+\s|http\S+\s', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text


In [5]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [6]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()
df.to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [7]:
df

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


### Add tokenized column

191210 정체불명의 error 발생
```
NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)
```

babynews 데이터가 추가된 후 갑자기 발생한 에러인데.. 왜인지 모르겠지만 해당 부분 데이터프레임만 따로 tokenize하는 것은 문제없이 돌아간다.  
따라서 번거롭지만 분리하여 tokenizing 한 후에 합치기로 하였다.  

-> 해결.. `NaN` value 문제였다(어이없게도 NaN 값이 중간에 섞여있었음;). 누락 데이터를 잘 체크하는 습관을 갖도록하자.

In [8]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text

In [9]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 1min 1s, sys: 30.4 s, total: 1min 32s
Wall time: 1min 32s


,text,tokenized
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성..."
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계..."
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ..."
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,..."
25348,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...,"[최근, 전자파, 가, 건강, 상, 의, 문제, 를, 일으킬, 수, 있, 다는, 견..."


In [10]:
df.isnull().sum()

text         0
tokenized    0
dtype: int64

## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [11]:
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [12]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [13]:
%%time
model_all = Word2Vec(corpus, size=128, workers=4, sg=1)
model_all.save('data/tokmom_word2vec_all.model')

CPU times: user 6min 35s, sys: 444 ms, total: 6min 35s
Wall time: 1min 44s


#### 유사어 분석

In [14]:
model_all.wv.similar_by_word("놀이")

[('소꿉', 0.7395849227905273),
 ('공놀이', 0.7213114500045776),
 ('기차놀이', 0.7077705264091492),
 ('숨바꼭질', 0.6862096786499023),
 ('보물찾기', 0.6849431991577148),
 ('낚시', 0.6765079498291016),
 ('손유희', 0.6695200800895691),
 ('샌드백', 0.6683435440063477),
 ('놀잇감', 0.6681313514709473),
 ('타악기', 0.6672809720039368)]

In [15]:
model_all.wv.similar_by_word("아이")

[('아기', 0.727118730545044),
 ('어린아이', 0.7252939939498901),
 ('갓난아이', 0.6651442050933838),
 ('윤우', 0.641637921333313),
 ('갓난아기', 0.6355990171432495),
 ('큰아이', 0.6338458061218262),
 ('경탄', 0.6180907487869263),
 ('엄마', 0.6158785223960876),
 ('놀림거리', 0.6148250699043274),
 ('친해진다', 0.6143154501914978)]

In [16]:
model_all.wv.similar_by_word("치아")

[('잇몸', 0.8324146270751953),
 ('영구치', 0.8024177551269531),
 ('충치', 0.7791770100593567),
 ('젖니', 0.7412041425704956),
 ('턱뼈', 0.7358801364898682),
 ('앞니', 0.7110280990600586),
 ('어금니', 0.701298713684082),
 ('법랑질', 0.6971782445907593),
 ('유치', 0.6962482929229736),
 ('치석', 0.6936749815940857)]

In [17]:
model_all.wv.similar_by_word("이빨")

[('혓바닥', 0.7846129536628723),
 ('거스러미', 0.7622735500335693),
 ('핥', 0.7522270679473877),
 ('오물거리', 0.750676155090332),
 ('영인', 0.7501369714736938),
 ('뜯어내', 0.7418147325515747),
 ('빨아먹', 0.7411083579063416),
 ('허옇', 0.7400315403938293),
 ('생채기', 0.7381551265716553),
 ('갈라져', 0.7378203868865967)]

In [18]:
model_all.wv.similar_by_word("감기")

[('코감기', 0.8325313329696655),
 ('목감기', 0.8247150182723999),
 ('기침감기', 0.7894113063812256),
 ('독감', 0.7855520844459534),
 ('냉방병', 0.7631239891052246),
 ('호흡기', 0.7490089535713196),
 ('비염', 0.7457585334777832),
 ('인후염', 0.7369061708450317),
 ('환절기', 0.7344949245452881),
 ('잔병', 0.7328144907951355)]

In [19]:
model_all.wv.similar_by_word("열")

[('고열로', 0.6223058700561523),
 ('미열', 0.5906575918197632),
 ('사병', 0.5777747631072998),
 ('오한', 0.5763400793075562),
 ('식혀', 0.5650386810302734),
 ('열기', 0.5645030736923218),
 ('고열', 0.563922643661499),
 ('허열', 0.5582061409950256),
 ('들어온다고', 0.5557551383972168),
 ('식히', 0.5507949590682983)]

In [20]:
model_all.wv.similar_by_word("아파")

[('아파서', 0.7917320728302002),
 ('아프', 0.7879322171211243),
 ('다쳐', 0.7685312032699585),
 ('칭얼댄다', 0.742912769317627),
 ('아파하', 0.7379513382911682),
 ('토해서', 0.7365566492080688),
 ('다쳐서', 0.7300858497619629),
 ('아파도', 0.7264070510864258),
 ('아픈데', 0.7248853445053101),
 ('고파도', 0.7156646251678467)]

In [21]:
model_all.wv.similar_by_word("아프")

[('아파', 0.7879322171211243),
 ('아파하', 0.7760176658630371),
 ('결리', 0.7617043256759644),
 ('쑤시', 0.7467668056488037),
 ('어지럽', 0.7446278929710388),
 ('아파서', 0.7411390542984009),
 ('거린', 0.7408324480056763),
 ('지끈지끈', 0.7363989949226379),
 ('간지럽', 0.7363499999046326),
 ('주물렀', 0.7318865060806274)]

In [22]:
model_all.wv.similar_by_word("울음")

[('터뜨립니다', 0.723886251449585),
 ('터뜨린다', 0.6939476728439331),
 ('터뜨릴', 0.6881331205368042),
 ('터뜨린', 0.6845754384994507),
 ('옹알이', 0.6715842485427856),
 ('우렁찬', 0.6670546531677246),
 ('어르', 0.6614100337028503),
 ('자지러질', 0.660844087600708),
 ('지르', 0.6574312448501587),
 ('고래고래', 0.6564108729362488)]

In [23]:
model_all.wv.similar_by_word("남아")

[('여아', 0.8627681136131287),
 ('남근기', 0.6519829630851746),
 ('남자', 0.6464985609054565),
 ('여자', 0.6393122673034668),
 ('엘렉트라', 0.6158814430236816),
 ('정상아', 0.614351749420166),
 ('018', 0.6046236753463745),
 ('경산부', 0.6019124984741211),
 ('1317', 0.601548433303833),
 ('아일', 0.5991610288619995)]

In [24]:
model_all.wv.similar_by_word("여아")

[('남아', 0.8627680540084839),
 ('여자', 0.644571840763092),
 ('남자', 0.6159909963607788),
 ('아든', 0.5759382247924805),
 ('남근기', 0.5696649551391602),
 ('아일', 0.5692425966262817),
 ('018', 0.5644198656082153),
 ('1317', 0.5590461492538452),
 ('인치', 0.5534515380859375),
 ('019', 0.5534112453460693)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자
- 유사한 키워드 잡기를 목적으로 할 때는 명사만 추출하는 것이 더 적절한 듯

In [25]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

CPU times: user 1min 3s, sys: 30.7 s, total: 1min 34s
Wall time: 1min 33s


In [26]:
noun_corpus = np.array(df.noun)

In [27]:
%%time
model_noun = Word2Vec(noun_corpus, size=100, workers=4, sg=1)
model_noun.save('data/tokmom_word2vec_only_noun.model')

CPU times: user 2min 52s, sys: 241 ms, total: 2min 52s
Wall time: 46.9 s


In [28]:
model_noun.wv.similar_by_word("육아")

[('워킹맘', 0.6665226221084595),
 ('양육', 0.6548275947570801),
 ('열혈', 0.6474577188491821),
 ('살림', 0.6366163492202759),
 ('독박', 0.6362835168838501),
 ('지원군', 0.6352605819702148),
 ('장세희', 0.6308539509773254),
 ('아서', 0.6258280277252197),
 ('바이블', 0.6200488805770874),
 ('대디', 0.6197760105133057)]

In [29]:
model_noun.wv.similar_by_word("시댁")

[('시부모', 0.8503279089927673),
 ('친정', 0.8500319123268127),
 ('본가', 0.8288648724555969),
 ('시어머니', 0.822963535785675),
 ('장모님', 0.8125674724578857),
 ('친정어머니', 0.8107408285140991),
 ('처가', 0.7997740507125854),
 ('대소사', 0.7902892231941223),
 ('남편', 0.7878108620643616),
 ('시누이', 0.7866990566253662)]

In [30]:
model_noun.wv.similar_by_word("놀이")

[('놀잇감', 0.7491439580917358),
 ('공놀이', 0.7258742451667786),
 ('소꿉', 0.715566098690033),
 ('무너뜨리기', 0.7084563970565796),
 ('누가누', 0.7057775855064392),
 ('샌드백', 0.7046945095062256),
 ('손인', 0.6978451609611511),
 ('사방치기', 0.6955163478851318),
 ('기차놀이', 0.6928429007530212),
 ('장난감', 0.685832679271698)]

In [31]:
model_noun.wv.similar_by_word("아이")

[('지레짐작', 0.7683221697807312),
 ('어린아이', 0.7581089735031128),
 ('엄마', 0.7560223340988159),
 ('낼', 0.744960367679596),
 ('과욕', 0.741504430770874),
 ('시기상조', 0.7410414218902588),
 ('걸', 0.7399687170982361),
 ('진득', 0.7355153560638428),
 ('소민', 0.7350615859031677),
 ('한가', 0.7345150709152222)]

In [32]:
model_noun.wv.similar_by_word("치아")

[('잇몸', 0.8670015335083008),
 ('영구치', 0.8479639887809753),
 ('충치', 0.8332496881484985),
 ('젖니', 0.8294187784194946),
 ('앞니', 0.7807421684265137),
 ('어금니', 0.7806175947189331),
 ('유치', 0.7572829723358154),
 ('법랑질', 0.7520350217819214),
 ('우식증', 0.7470067739486694),
 ('턱뼈', 0.7460055947303772)]

In [33]:
model_noun.wv.similar_by_word("이빨")

[('혓바닥', 0.7857290506362915),
 ('영인', 0.7829234600067139),
 ('부리', 0.7599800825119019),
 ('칼자국', 0.7565423846244812),
 ('문다', 0.7542744874954224),
 ('골짜기', 0.7500026226043701),
 ('나방', 0.7496861219406128),
 ('밥풀', 0.7489989995956421),
 ('거스러미', 0.7414813041687012),
 ('꽁지', 0.7374645471572876)]

In [34]:
model_noun.wv.similar_by_word("감기")

[('목감기', 0.8112162351608276),
 ('코감기', 0.8103852272033691),
 ('환절기', 0.807023286819458),
 ('편도염', 0.7916395664215088),
 ('기관지염', 0.7842141389846802),
 ('기침감기', 0.7825652360916138),
 ('기침', 0.7747663259506226),
 ('호흡기', 0.7688245177268982),
 ('콧물', 0.7670191526412964),
 ('비염', 0.7587753534317017)]

In [35]:
model_noun.wv.similar_by_word("열")

[('고열로', 0.7383891344070435),
 ('불덩이', 0.7031384706497192),
 ('熱', 0.7006406188011169),
 ('외감', 0.6992070078849792),
 ('오한', 0.6978108882904053),
 ('사병', 0.6920907497406006),
 ('고열', 0.6916845440864563),
 ('허열', 0.6879662275314331),
 ('양허', 0.6745286583900452),
 ('인동', 0.6729408502578735)]

In [36]:
model_noun.wv.similar_by_word("아파")

[('거립니다', 0.8415960073471069),
 ('피멍', 0.8308123350143433),
 ('참기', 0.8229003548622131),
 ('불덩이', 0.8130450248718262),
 ('청진', 0.8058290481567383),
 ('무의식중', 0.8047473430633545),
 ('소름', 0.7994725704193115),
 ('어린애', 0.7992778420448303),
 ('생채기', 0.798244833946228),
 ('거려', 0.7976727485656738)]

In [37]:
model_noun.wv.similar_by_word("울음")

[('소리', 0.7069542407989502),
 ('비명', 0.70014488697052),
 ('놀람', 0.6990275382995605),
 ('아달라', 0.6939420104026794),
 ('괴성', 0.6863733530044556),
 ('신음', 0.6844191551208496),
 ('큰소리', 0.6816563010215759),
 ('고음', 0.6803909540176392),
 ('지르기', 0.6771337985992432),
 ('옹알이', 0.6618772745132446)]

In [38]:
model_noun.wv.similar_by_word("남아", topn=20)

[('여아', 0.8808973431587219),
 ('변성기', 0.7365071177482605),
 ('액모', 0.7279012203216553),
 ('음모', 0.6897344589233398),
 ('클리토리스', 0.6826943159103394),
 ('여자', 0.6781058311462402),
 ('정류', 0.6751503944396973),
 ('기라', 0.6682007312774658),
 ('남자', 0.6680871248245239),
 ('남근기', 0.6527450084686279),
 ('몽우리', 0.6477795839309692),
 ('대음순', 0.6453688144683838),
 ('아도', 0.6425366401672363),
 ('생시', 0.6357797384262085),
 ('증후', 0.635003387928009),
 ('음경', 0.6348810791969299),
 ('조숙', 0.6296437978744507),
 ('망연자실', 0.6274477243423462),
 ('초경', 0.626950204372406),
 ('색맹', 0.6255058646202087)]

In [39]:
model_noun.wv.similar_by_word("여아", topn=20)

[('남아', 0.8808972239494324),
 ('여자', 0.6845464110374451),
 ('변성기', 0.6617922782897949),
 ('액모', 0.655515730381012),
 ('기라', 0.6520875692367554),
 ('음모', 0.6462687253952026),
 ('남자', 0.6460123062133789),
 ('남근기', 0.6342167854309082),
 ('클리토리스', 0.6258744597434998),
 ('성별', 0.6191517114639282),
 ('가슴둘레', 0.6170589327812195),
 ('김우주', 0.6141172051429749),
 ('남학생', 0.6068969964981079),
 ('생시', 0.6026173233985901),
 ('초경', 0.6009048223495483),
 ('몽우리', 0.5905047655105591),
 ('항문기', 0.5886027812957764),
 ('아도', 0.5874003767967224),
 ('대음순', 0.5848087072372437),
 ('임소', 0.5836001038551331)]

In [40]:
model_noun.wv.similar_by_word("여자", topn=20)

[('남자', 0.9376375079154968),
 ('금성', 0.7248766422271729),
 ('조신', 0.7210137844085693),
 ('남탕', 0.7042510509490967),
 ('족쇄', 0.6975350379943848),
 ('여탕', 0.6964397430419922),
 ('성차', 0.6901171207427979),
 ('여아', 0.6845464706420898),
 ('아이든', 0.6811325550079346),
 ('남아', 0.6781058311462402),
 ('남학생', 0.6677274107933044),
 ('엘렉트라', 0.6644693613052368),
 ('바람둥이', 0.6463067531585693),
 ('얌전', 0.6390299797058105),
 ('남근', 0.6332674622535706),
 ('태사혜', 0.6304758787155151),
 ('군대', 0.6293023228645325),
 ('이방인', 0.6286194324493408),
 ('시집살이', 0.6270216107368469),
 ('에너자이저', 0.6230416297912598)]

In [41]:
model_noun.wv.similar_by_word("남자", topn=20)

[('여자', 0.9376375675201416),
 ('금성', 0.7521324157714844),
 ('조신', 0.7450736165046692),
 ('족쇄', 0.7341678142547607),
 ('남탕', 0.7237294316291809),
 ('여탕', 0.7156965732574463),
 ('성차', 0.7060838937759399),
 ('얌전', 0.7016324400901794),
 ('아이든', 0.6778429746627808),
 ('엘렉트라', 0.6766166687011719),
 ('위엄', 0.6684761047363281),
 ('남아', 0.6680871248245239),
 ('남근', 0.6679927706718445),
 ('잠지', 0.6597963571548462),
 ('야동', 0.6576305031776428),
 ('이방인', 0.6543021202087402),
 ('바람둥이', 0.6532995700836182),
 ('남학생', 0.6478467583656311),
 ('알파걸', 0.6465926170349121),
 ('우월주의', 0.6462047100067139)]

In [42]:
model_noun.wv.similar_by_word("유모차")

[('디럭스', 0.8062554597854614),
 ('절충', 0.7802572250366211),
 ('타보', 0.7736871242523193),
 ('휴대용', 0.7510476112365723),
 ('핸들링', 0.7220766544342041),
 ('퀴니', 0.7042291164398193),
 ('쇼퍼', 0.7030842304229736),
 ('맥클라렌', 0.6988304853439331),
 ('잉글', 0.6975149512290955),
 ('페도라', 0.6926717758178711)]

In [43]:
model_noun.wv.similar_by_word("카시트")

[('브라이', 0.7573115825653076),
 ('안전벨트', 0.7391207814216614),
 ('콩코드', 0.7227051854133606),
 ('타카타', 0.7216581106185913),
 ('컨버터블', 0.7101017236709595),
 ('뒷좌석', 0.7100356817245483),
 ('순성', 0.7090908288955688),
 ('점식', 0.7000877857208252),
 ('장착', 0.6985743045806885),
 ('부스터', 0.6983839273452759)]

### Volume of Corpus 

In [44]:
df[['text']].to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [45]:
pd.read_csv('data/text_for_word2vec.csv')

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


### Loading check

In [46]:
model = Word2Vec.load('data/tokmom_word2vec_only_noun.model')
model

## Doc2Vec

In [47]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [48]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['noun'])]

In [49]:
%%time
doc2vec_model = Doc2Vec(documents, 
                        vector_size=100, 
                        window=5, 
                        min_count=5, 
                        workers=4)

model.save('data/tokmom_doc2vec_only_noun.model')

CPU times: user 1min 4s, sys: 1.35 s, total: 1min 5s
Wall time: 27.4 s


In [50]:
len(doc2vec_model.docvecs)

25349

In [51]:
df['text'][0]

'6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다'

In [52]:
print("Input Document", df['text'][100], sep="\n")

similar_docs = doc2vec_model.docvecs.most_similar(100)

print("Most similar Documents")
for idx, similarity in similar_docs[:3]:
    print("idx:", idx, "similarity:", np.round(similarity, 3))
    print(df['text'][idx])

Input Document
젖병을 떼야 하는 이유로는 첫째 우식증을 예방하고 둘째 젖병 무는 어린 아기의 습관을 서서히 고쳐 나가기 위함 입니다 지금은 단지 습관에 의해 젖병을 물고 있을 뿐이므로 음식은 숟가락으로 먹이고 우유는 컵으로 마시게 하면서 젖병을 뗄 수 있도록 해야 합니다 잠잘 때 젖병을 물고 자는 것은 무의식적인 행동입니다 그러나 너무 친숙해진 습관을 갑자기 금지시킨다면 아기 심리에 큰 충격을 줄 수도 있으므로 서서히 젖병 속의 우유를 희석시키는 방법을 사용합니다 예를 들어 평상시에 우유를 200 mL 먹였다면 우유 180 mL에 물 20 mL를 섞어서 젖병에 넣어줍니다 점차 물의 농도를 높여가다가 아기가 이를 느끼고 부정적인 반응을 보일 때는 잠시 정지하였다가 다시 희석의 강도를 높여봅니다 그러다 보면 아기는 점차 희석된 우유에 흥미를 잃어 스스로 젖병을 내던지게 될 것입니다 아기에 따라 스스로 정서적 안정을 얻는 습관을 하나씩 가지고 있습니다 아기 때의 이불을 늘 잡고 있다거나 엄마의 머리카락을 잡아야 잠을 잔다거나 하는 습관은 고치기가 쉽지 않은 일입니다 젖병도 그런 습관의 일종일 수 있습니다 젖병을 물고 있는 것이 정서적인 이유라면 젖병 떼기를 강요하지 않습니다 하루 2 3번은 젖병을 물고 있게 하다가 점차 아기가 성장하면서 젖병 무는 것이 자신에게는 어울리지 않는다는 것을 엄마가 끊임없이 설득하고 아이 스스로도 그렇게 느껴 자연스레 젖병을 멀리하게 합니다 가능한 컵이나 빨대컵을 활용하고 노리개 젖꼭지 등을 대체하여 사용해 볼 수도 있습니다
Most similar Documents
idx: 15222 similarity: 0.681
7개월 된 여자 아기인데 생후 3개월 되던 때부터 엄지손가락을 빨기 시작하더니 요즘에는 젖병을 입에서 떼기가 무섭게 손가락을 가져가요 손가락을 빼면 화를 내고 이리저리 고개를 돌려 손가락을 찾고 빨아야만 잠이 듭니다 저희 아기는 생후 50일도 안 돼서 주먹을 빨더니 요즘은 손가락을 넣고 쩝쩝 소리까지 내가며 맛있

In [53]:
df['tokenized_doc'] = df['noun'].apply(lambda x: doc2vec_model.infer_vector(x).astype(float))

In [54]:
df

,text,tokenized,noun,tokenized_doc
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,...","[0.0666288509964943, -0.2398962378501892, 0.08..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ...","[-0.031621795147657394, -0.1197349950671196, 0..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외...","[-0.03517955541610718, 0.06947168707847595, -0..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일...","[-0.23605751991271973, -0.10074695944786072, -..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ...","[-0.031331680715084076, -0.05370507389307022, ..."
...,...,...,...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[-0.058275945484638214, -0.8878468871116638, 0..."
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계...","[아토피, 유전, 요인, 작용, 사실, 유전, 요인, 뿐, 아토피, 유발, 수, 하...","[0.3867766559123993, 0.3105510473251343, -0.13..."
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 것, 균형, 영양, 음식, 섭취, 태...","[-0.025240298360586166, -0.9818443655967712, 0..."
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,...","[분만, 여성, 수, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[-0.27099451422691345, -0.16602614521980286, 0..."
